In [2]:
# Imports
import pandas as pd

In [107]:
# This is where we load all the data through oracle; but for now I'll just read.csv the data in.

emissions = pd.read_csv("data/tidied_emissions_with_key.csv")
employment = pd.read_csv("data/tidied_employment_estimates.csv")
industry = pd.read_csv("data/tidied_industry_mapping.csv")
population = pd.read_csv("data/tidied_population_estimates.csv")

In [264]:
## model1: emissions/employed (by year, province) ~ good type(dummy) + province (dummy) + year

# joining emissions and industry together
emissions_industry = pd.merge(emissions, industry, left_on="industry_code", right_on="IndustryCode")

# filter out canada in emissions_industry dataset
emissions_industry = emissions_industry[emissions_industry["province"] != "Canada"]

# mutate a new column called ProvinceYearDurable in EmissionsIndustry
emissions_industry["ProvinceYearDurable"] = emissions_industry["province"] + emissions_industry["year"].astype(str) + emissions_industry["Durable"].astype(str)

# remove households, put in separate dataframe
emissions_household = emissions_industry[emissions_industry["ioic_code"].isin(['9999999', '9999998'])]
emissions_industry = emissions_industry[~emissions_industry["ioic_code"].isin(['9999999', '9999998'])]

# drop a few duplicate columns
emissions_industry.drop(columns=["IndustryCode", "IndustryName", "IOIC", "industry_name", "industry_code", "NAICS"], inplace=True)

# rename count to "emissions"
emissions_industry.rename(columns={'count': 'emissions'}, inplace=True)

# group by Province, Year, Durable, sum emissions for each industry
grouped_emissions = pd.DataFrame(emissions_industry.groupby(['province', 'year', 'Durable'])['emissions'].sum())
grouped_emissions = grouped_emissions.reset_index()
grouped_emissions["ProvinceYearDurable"] = grouped_emissions["province"] + grouped_emissions["year"].astype(str) + grouped_emissions["Durable"].astype(str)




# remove Manufacturing, Canada from naics_industry
employment_tidy = employment[(employment["naics_industry"] != "Manufacturing") & (employment["province"] != "Canada")]

# rename employment estimate data's manufacturing column to 0 or 1
employment_tidy['durable'] = employment_tidy['naics_industry'].map({'Durables': 1, 'Non-durables': 0})
employment_tidy = employment_tidy.drop(columns=['naics_industry'])

# mutate a new column called ProvinceYearDurable in employment
employment_tidy["ProvinceYearDurable"] = employment_tidy["province"] + employment_tidy["year"].astype(str) + employment_tidy["durable"].astype(str)

# remove Manufacturing, Canada from naics_industry
employment_tidy = employment[(employment["naics_industry"] != "Manufacturing") & (employment["province"] != "Canada")]

# rename employment estimate data's manufacturing column to 0 or 1
employment_tidy['durable'] = employment_tidy['naics_industry'].map({'Durables': 1, 'Non-durables': 0})
employment_tidy = employment_tidy.drop(columns=['naics_industry'])

# mutate a new column called ProvinceYearDurable in employment
employment_tidy["ProvinceYearDurable"] = employment_tidy["province"] + employment_tidy["year"].astype(str) + employment_tidy["durable"].astype(str)
employment_tidy["employed"] = employment_tidy["count"]
employment_tidy = employment_tidy.drop(columns=["count", "province", "year", "durable"])




# join the grouped_emissions with employment_tidy, drop key
emissions_industry_employment = pd.merge(grouped_emissions, employment_tidy, on="ProvinceYearDurable")
emissions_industry_employment = emissions_industry_employment.drop(columns="ProvinceYearDurable")


/tmp/ipykernel_156/3112970236.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  employment_tidy['durable'] = employment_tidy['naics_industry'].map({'Durables': 1, 'Non-durables': 0})
/tmp/ipykernel_156/3112970236.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  employment_tidy['durable'] = employment_tidy['naics_industry'].map({'Durables': 1, 'Non-durables': 0})


In [265]:
## model 2: emissions/population (by year, province) ~ province(dummy) * year ---> households

# drop a few useless columns
emissions_household = emissions_household.drop(columns=["industry_code", "IndustryCode", "IOIC", "Durable", "ProvinceYearDurable"

,province,industry_name,ioic_code,year,count,industry_code,IndustryCode,IndustryName,NAICS,IOIC,Durable,ProvinceYearDurable
10,Newfoundland and Labrador,Households: Motor fuels and lubricants,9999999,2012,1742,99999,99999,Households: Motor fuels and lubricants,999,9999999,0,Newfoundland and Labrador20120
11,Newfoundland and Labrador,Households: Motor fuels and lubricants,9999999,2013,1711,99999,99999,Households: Motor fuels and lubricants,999,9999999,0,Newfoundland and Labrador20130
12,Newfoundland and Labrador,Households: Motor fuels and lubricants,9999999,2014,1926,99999,99999,Households: Motor fuels and lubricants,999,9999999,0,Newfoundland and Labrador20140
13,Newfoundland and Labrador,Households: Motor fuels and lubricants,9999999,2015,1897,99999,99999,Households: Motor fuels and lubricants,999,9999999,0,Newfoundland and Labrador20150
14,Newfoundland and Labrador,Households: Motor fuels and lubricants,9999999,2016,1809,99999,99999,Households: Motor fuels and lubricants,999,9999999,0,Newfoundland and Labrador20160
...,...,...,...,...,...,...,...,...,...,...,...,...
275,Nunavut,Households: Electricity and other fuels,9999998,2017,29,99998,99998,Households: Electricity and other fuels,998,9999998,0,Nunavut20170
276,Nunavut,Households: Electricity and other fuels,9999998,2018,24,99998,99998,Households: Electricity and other fuels,998,9999998,0,Nunavut20180
277,Nunavut,Households: Electricity and other fuels,9999998,2019,19,99998,99998,Households: Electricity and other fuels,998,9999998,0,Nunavut20190
278,Nunavut,Households: Electricity and other fuels,9999998,2020,14,99998,99998,Households: Electricity and other fuels,998,9999998,0,Nunavut20200


# linear regression: 

models:

1. emissions/population (by year, province) ~ province(dummy) * year ---> households
2. emissions/employed (by year, province) ~ good type(dummy) + province (dummy) + year